The following tutorial is available on the [Wallaroo Github Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20231004-wallaroo-inference-server/wallaroo-inference-server-tutorials/wallaroo-inference-server-hf-summarizer).

## Wallaroo Inference Server:  Faster R-CNN

This notebook is used in conjunction with the [Wallaroo Inference Server Free Edition](https://docs.wallaroo.ai/wallaroo-inferencing-server/) for the Computer Vision resnet 50 machine learning model.  This provides a free license for performing inferences through the Computer Vision Faster R-CNN machine learning model.  For full demonstrations of this model, see [Wallaroo Use Case Tutorials](https://docs.wallaroo.ai/wallaroo-use-case-tutorials/wallaroo-use-case-computer-vision/use-case-computer-vision-retail/).

### Prerequisites

* A deployed Wallaroo Inference Server Free Edition with one of the following options:
  * **Wallaroo.AI HF ResNet50 Inference Server - x64**
* Access via port 8080 to the Wallaroo Inference Server Free Edition.

## Wallaroo Inference Server API Endpoints

The following HTTPS API endpoints are available for Wallaroo Inference Server.

### Pipelines Endpoint

* Endpoint: HTTPS GET `/pipelines`
* Returns:
  * List of `pipelines` with the following fields.
    * **id** (*String*): The name of the pipeline.
    * **status** (*String*): The pipeline status.  `Running` indicates the pipeline is available for inferences.

#### Pipeline Endpoint Example

The following demonstrates using `curl` to retrieve the Pipelines endpoint.  Replace the HOSTNAME with the address of your Wallaroo Inference Server.

In [21]:
!curl doc-example-cv-frcnn.westus2.cloudapp.azure.com:8080/pipelines

{"pipelines":[{"id":"frcnn","status":"Running"}]}

### Models Endpoint

* Endpoint: GET `/models`
* Returns:
  * List of `models` with the following fields.
    * **name** (*String*):  The name of the model.
    * **sha** (*String*):  The `sha` hash of the model.
    * **status** (*String*):  The model status.  `Running` indicates the models is available for inferences.
    * **version** (*String*): The model version in UUID format.

#### Models Endpoint Example

The following demonstrates using `curl` to retrieve the Models endpoint.  Replace the HOSTNAME with the address of your Wallaroo Inference Server.

In [22]:
!curl doc-example-cv-frcnn.westus2.cloudapp.azure.com:8080/models

{"models":[{"name":"frcnn","sha":"ee606dc9776a1029420b3adf59b6d29395c89d1d9460d75045a1f2f152d288e7","status":"Running","version":"0762d591-7d31-4738-8394-2a148d00fbdc"}]}

### Inference Endpoint

The inference endpoint takes the following pattern:

* `/pipelines/{pipeline-name}`:  The `pipeline-name` is the same as returned from the [`/pipelines`](#list-pipelines) endpoint as `id`.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing]({{<ref "wallaroo-sdk-essentials-pipeline#anomaly-testing">}}).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.


### Inference Endpoint Example

The following example performs an inference using the Apache Arrow table input `./data/image_224x224.arrow` from an image converted into a `tensor` for inferencing.

In [23]:
!curl -X POST doc-example-cv-frcnn.westus2.cloudapp.azure.com:8080/pipelines/frcnn \
    -H "Content-Type:application/vnd.apache.arrow.file" \
    --data-binary @./data/test_table.arrow

[{"check_failures":[],"elapsed":[62382994,3894096061],"model_name":"frcnn","model_version":"0762d591-7d31-4738-8394-2a148d00fbdc","original_data":null,"outputs":[{"Float":{"data":[2.1511011123657227,193.98316955566406,76.26535034179688,475.4029846191406,610.822509765625,98.60633087158203,639.8867797851562,232.27053833007812,544.2866821289062,98.7265396118164,581.2883911132812,230.20494079589844,454.9934387207031,113.0856704711914,484.7846374511719,210.12820434570312,502.5888671875,331.87664794921875,551.2268676757812,476.4918212890625,538.5425415039062,292.1205139160156,587.4655151367188,468.1288146972656,578.5416870117188,99.70755767822266,617.2246704101562,233.57081604003906,548.552001953125,191.84564208984375,577.3058471679688,238.4773712158203,459.8332824707031,344.297119140625,505.42633056640625,456.7117919921875,483.4716796875,110.56584930419922,514.0936279296875,205.00155639648438,262.1221923828125,190.36659240722656,323.49029541015625,405.2057800292969,511.6675109863281,104.538